# Download landsat scenes over the San Juan Islands

This notebook demonstrates:
* Using the pystac client to retrieve images from microsofts cloud storage
* Saving those images to netcdf

Code adapted from: https://medium.com/@geonextgis/getting-started-with-microsoft-planetary-computer-stac-api-67cbebe96e5e

In [3]:
import pystac_client
import planetary_computer
import odc.stac
from pystac.extensions.eo import EOExtension as eo

/home/caprice/micromamba/envs/summerfog/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [41]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)

#opening a connection to the STAC API endpoint hosted by the Planetary Computer platform
#check website for more info


In [44]:
# FHL location on San Juans
bbox_of_interest = [-123.33711789011167, 48.4, -122.65866282644907, 48.93242519875351]
#times_of_interest = "2022-06-01/2022-06-30"
times_of_interest = [
    "2022-07-27/2022-07-27",
    "2022-08-04/2022-08-04",
    "2022-08-12/2022-08-12",
    "2022-08-20/2022-08-20",
    "2022-09-29/2022-09-29",
    "2021-06-30/2021-07-01",
]
#these are all the good dates from summer2022, listed in word doc from jessica

this cell finds all the scenes that match the bbox_of_interest and times_of_interest

In [45]:
items_list = []
for time_stamp in times_of_interest:   #goes through times in list above
    # Query the catalog
    search = catalog.search(
        collections=["landsat-c2-l2", "landsat-c1-l2"],
        bbox=bbox_of_interest,
        datetime=time_stamp,
        # #  Uncomment the two lines below to sort the image collection based on 'cloud_cover'
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
    )

    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    items_list.append(items)

Returned 1 Items
Returned 1 Items
Returned 2 Items
Returned 0 Items
Returned 1 Items
Returned 3 Items


In [43]:
items_list[1][1]

<Item id=LC08_L2SP_047026_20220804_02_T1>

landsat visits a location every 16 days, 
this cell goes through all the scenes found above and downloads them

In [46]:
datas = []
for items in items_list:
    for selected_item in items:
        try:
            data = odc.stac.stac_load(
                [selected_item], bbox=bbox_of_interest
            ).isel(time=0)
            fn = f"{str(data.time.values)}.nc" 
            print(fn)
            data.to_netcdf(fn)
        except Exception as e: 
            print(f"Failed on {selected_item} with error: {e}")

2022-07-27T19:01:26.703735000.nc


Aborting load due to failure while reading: https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2022/047/026/LC08_L2SP_047026_20220804_20220818_02_T1/LC08_L2SP_047026_20220804_20220818_02_T1_ST_URAD.TIF?st=2025-06-23T17%3A45%3A37Z&se=2025-06-24T18%3A30%3A37Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-06-24T10%3A52%3A31Z&ske=2025-07-01T10%3A52%3A31Z&sks=b&skv=2024-05-04&sig=xm2/MlmIBhnUifhEbj2VX1lEPcqVNijIkNqJ5FRiKRQ%3D:1


Failed on <Item id=LC08_L2SP_047026_20220804_02_T1> with error: '/vsicurl/https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2022/047/026/LC08_L2SP_047026_20220804_20220818_02_T1/LC08_L2SP_047026_20220804_20220818_02_T1_ST_URAD.TIF?st=2025-06-23T17%3A45%3A37Z&se=2025-06-24T18%3A30%3A37Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-06-24T10%3A52%3A31Z&ske=2025-07-01T10%3A52%3A31Z&sks=b&skv=2024-05-04&sig=xm2/MlmIBhnUifhEbj2VX1lEPcqVNijIkNqJ5FRiKRQ%3D' not recognized as being in a supported file format.
2022-08-12T19:01:36.957727000.nc
2022-08-12T17:32:24.353686000.nc
2022-09-29T19:01:47.715049000.nc
2021-07-01T17:59:40.670009000.nc
2021-07-01T17:59:16.791493000.nc
2021-06-30T19:01:19.764388000.nc


In [ ]:

selected_item = items_list[1][1]
        try:
            data = odc.stac.stac_load(
                [selected_item], bbox=bbox_of_interest
            ).isel(time=0)
            fn = f"{str(data.time.values)}.nc" 
            print(fn)
            data.to_netcdf(fn)
        except Exception as e: 
            print(f"Failed on {selected_item} with error: {e}")

In [14]:
data

<xarray.Dataset> Size: 145MB
Dimensions:      (y: 2335, x: 1678, time: 1)
Coordinates:
  * y            (y) float64 19kB 5.42e+06 5.42e+06 ... 5.35e+06 5.35e+06
  * x            (x) float64 13kB 4.75e+05 4.75e+05 ... 5.253e+05 5.253e+05
    spatial_ref  int32 4B 32610
  * time         (time) datetime64[ns] 8B 2022-07-27T19:01:26.703735
Data variables: (12/19)
    qa           (time, y, x) int16 8MB 260 259 259 259 259 ... 307 308 308 305
    red          (time, y, x) uint16 8MB 8225 8189 8150 ... 10080 10031 9677
    blue         (time, y, x) uint16 8MB 8397 8351 8318 8374 ... 8831 8901 8704
    drad         (time, y, x) int16 8MB 1373 1373 1373 1373 ... 1388 1388 1388
    emis         (time, y, x) int16 8MB 9880 9880 9880 9880 ... 9645 9645 9712
    emsd         (time, y, x) int16 8MB 0 0 0 0 0 0 ... 128 128 128 128 128 106
    ...           ...
    swir16       (time, y, x) uint16 8MB 7352 7328 7334 ... 11820 11348 11780
    swir22       (time, y, x) uint16 8MB 7382 7356 7359 ... 10278 10137 10305
    coastal      (time, y, x) uint16 8MB 7955 7935 7889 7949 ... 8525 8611 8390
    qa_pixel     (time, y, x) uint16 8MB 21952 21952 21952 ... 21824 21824 21824
    qa_radsat    (time, y, x) uint16 8MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    qa_aerosol   (time, y, x) uint8 4MB 164 164 164 164 164 ... 96 96 96 96 160

In [12]:
data

<xarray.Dataset> Size: 145MB
Dimensions:      (y: 2335, x: 1678)
Coordinates:
  * y            (y) float64 19kB 5.42e+06 5.42e+06 ... 5.35e+06 5.35e+06
  * x            (x) float64 13kB 4.75e+05 4.75e+05 ... 5.253e+05 5.253e+05
    spatial_ref  int32 4B 32610
    time         datetime64[ns] 8B 2022-07-27T19:01:26.703735
Data variables: (12/19)
    qa           (y, x) int16 8MB 260 259 259 259 259 ... 307 307 308 308 305
    red          (y, x) uint16 8MB 8225 8189 8150 8213 ... 9579 10080 10031 9677
    blue         (y, x) uint16 8MB 8397 8351 8318 8374 ... 8593 8831 8901 8704
    drad         (y, x) int16 8MB 1373 1373 1373 1373 ... 1388 1388 1388 1388
    emis         (y, x) int16 8MB 9880 9880 9880 9880 ... 9776 9645 9645 9712
    emsd         (y, x) int16 8MB 0 0 0 0 0 0 0 ... 121 128 128 128 128 128 106
    ...           ...
    swir16       (y, x) uint16 8MB 7352 7328 7334 7358 ... 11820 11348 11780
    swir22       (y, x) uint16 8MB 7382 7356 7359 7379 ... 10278 10137 10305
    coastal      (y, x) uint16 8MB 7955 7935 7889 7949 ... 8361 8525 8611 8390
    qa_pixel     (y, x) uint16 8MB 21952 21952 21952 21952 ... 21824 21824 21824
    qa_radsat    (y, x) uint16 8MB 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    qa_aerosol   (y, x) uint8 4MB 164 164 164 164 164 164 ... 96 96 96 96 96 160